In [108]:
import numpy as np
from scipy.optimize import minimize
from astropy.convolution import convolve, Box2DKernel

In [51]:
def get_digit(number, n):
    return number // 10**n % 10

In [171]:
size = 300
grid_serial = 6303
grid = np.ones((size, size))
y, x = np.mgrid[0:size, 0:size]
rack_id = x + 10
power_level = rack_id * y
power_level += grid_serial
power_level *= rack_id
power_level = get_digit(power_level, 2) - 5

boxsize = 14
kern = Box2DKernel(boxsize)
ravel_max = np.argmax(convolve(power_level, kern))

center_y, center_x = np.unravel_index(ravel_max, dims=(size, size))
corner_x = int(center_x + 0.5 - boxsize/2)
corner_y = int(center_y + 0.5 - boxsize/2)
corner_x, corner_y, ravel_max

(284, 50, 17391)

In [166]:
def min_func(s):
    global power_level
    global size
    kern = Box2DKernel(s)
    max_1d = np.argmax(convolve(power_level, kern))
    if s > size:
        s = size
    if s < 1:
        s = 1
    yc, xc = np.unravel_index(max_1d, dims=(size, size))
    lx, ly = int(xc + 0.5 - s/2), int(yc + 0.5 - s/2)
    ux, uy = lx+s+1, ly+s+1  # numpy quirk to slice one past where you want
    power = power_level[ly:uy, lx:ux].sum()  # set negative to minimize
    return power

In [165]:
pars = (3)
min_results = minimize(min_func, pars)
best_size = np.round(min_results['x'])
min_results

      fun: -27
 hess_inv: array([[1]])
      jac: array([0.])
  message: 'Optimization terminated successfully.'
     nfev: 3
      nit: 0
     njev: 1
   status: 0
  success: True
        x: array([3.])

In [169]:
powers = []
for bs in range(1,int(size/2),1):
    powers.append(min_func(bs))
powers

[-4,
 23,
 20,
 49,
 52,
 64,
 52,
 63,
 50,
 49,
 61,
 71,
 63,
 73,
 49,
 60,
 65,
 72,
 67,
 52,
 31,
 41,
 22,
 20,
 25,
 12,
 18,
 2,
 -15,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -630,
 0,
 0,
 0,
 -683,
 -717,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -851,
 -851,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -1287,
 -1333,
 -1320,
 -1347,
 -1347,
 0,
 0,
 0,
 0,
 0,
 -1573,
 -1617,
 -1574,
 -1640,
 -1598,
 -1677,
 -1677,
 -1840,
 -1840,
 -1929,
 -1929,
 -2007,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 -2547,
 -2547,
 -2618,
 -2604,
 -2644,
 -2644]

In [183]:
power_level[0:3, 0:3]

array([[-5, -2,  1],
       [-4, -1,  2],
       [-3,  0,  4]])

In [181]:
for i in range(5):
    for j in range(5):
        lx, ly = 284+i-2, 50+j-2
        ux, uy = lx+boxsize+1, ly+boxsize+1
        psum = power_level[ly:uy, lx:ux].sum()
        print(lx, ly, psum)

282 48 31
282 49 54
282 50 48
282 51 32
282 52 46
283 48 40
283 49 61
283 50 63
283 51 45
283 52 56
284 48 45
284 49 69
284 50 73
284 51 47
284 52 50
285 48 38
285 49 59
285 50 61
285 51 41
285 52 41
286 48 6
286 49 23
286 50 22
286 51 8
286 52 14


In [203]:
# heck with it, go straight brute force...
pmaxes = []
xmaxes = []
ymaxes = []
for s in range(1, size, 1):
    pmax = -np.inf
    for i in range(size-s+1):
        for j in range(size-s+1):
            p = power_level[i:i+s, j:j+s].sum()
            if p > pmax:
                pmax = p
                xmax = j
                ymax = i
    pmaxes.append(pmax)
    xmaxes.append(xmax)
    ymaxes.append(ymax)

In [204]:
np.argmax(pmaxes)

11

In [205]:
xmaxes[11], ymaxes[11]

(284, 172)

In [206]:
pmaxes[11]

88

In [202]:
list(range(1, size, 1))[11]

12